#                Stock Market Technical and Fundamental Analysis








##  Loading in packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt





## Data Wrangling and joining data sets



## Loading in the institutional data set

#### # Please check out the google drive link for Data dictionary and other project maaterials

In [ ]:
institutions = pd.read_csv("D:/Institutions.csv", encoding='latin-1')

In [ ]:
institutions.shape

In [ ]:
institutions.columns

In [ ]:
# Let's see how many US stocks are in the data set

# CUSIP  code for us stocks contains only numbers

#num_cusip = []

#for i in range(0,len(institutions)):
#        num_cusip.append(str.isdigit(str(institutions['CUSIP'][i]))) #I fianlly get it. Append makes a big diff sometimes

In [ ]:
#only_US = institutions[num_cusip]

#len(only_US['CUSIP'].unique())

# 8899 stocks is a lot. Its safe to assume we have institutional data for all US stocks. we are going to use this data set then

In [ ]:
#only_US.shape




## Loading in the financials data set

In [ ]:
financials = pd.read_csv("D:/Financials.csv", encoding='latin-1')

In [ ]:
financials.shape

# So the financials data is nearly 3.5 times bigger than institutional data

In [ ]:
financials.columns

In [ ]:
# i wonder if it is possible to join financials and institutions data sets on CUSIP and date?

financials['DATE'][1:5]

In [ ]:
institutions['quarter'][1:5]

In [ ]:
institutions['rquarter'][1:5]

In [ ]:
financials['month1'] = financials['DATE'].astype(str).str[4]
financials['month2'] = financials['DATE'].astype(str).str[5]
financials['month'] = financials['month1'] + financials['month2']

In [ ]:
financials2 = financials
financials2['month'] = financials2['month'].astype(str).astype(int)
typical_quarter_end = [3,6,9,12]
financials2 = financials2.loc[financials2['month'].isin(typical_quarter_end)]
financials2.shape

# willing to ignore that 16% of data. Will decide later. With the systems I am trying to build and currently use, Portfolio's can be shuffled only 4 times. Can't make exceptions for some firms. moreover, most big firms follow the typical quarter ending schedule

In [ ]:
financials['year'] = financials['DATE'].astype(str)
financials['year'] = financials.year.str[0:4]
financials['month'] = financials['month'].astype(str)
financials['date_modified'] = financials['month'] + financials['year']
financials['date_modified'][1:5]

In [ ]:
institutions['month'] = institutions['rquarter'].astype(str)
institutions['month'] = institutions.rquarter.str[0:2]

In [ ]:
institutions['year'] = institutions['rquarter'].astype(str)
institutions['year'] = institutions.rquarter.str[6:11]

In [ ]:
institutions['date_modified'] = institutions['month'] + institutions['year']

In [ ]:
institutions['date_modified'][1:5]

In [ ]:
# forming the real keys to be joined on

financials['key1'] = financials['date_modified'] + financials['CUSIP']

In [ ]:
financials['key1'][1:5]

# If i remember right, there are some canadian stocks also in the financials set

In [ ]:
financials['EXCOUNTRY'].unique()

# I just relaized that I have data for 53 countries. What a shame we do not have price data. I can scrape but let me just focus on american stocks for now

In [ ]:
usa_markets = ['NAS', 'NYS', 'BATS']

financials_us = financials.loc[financials['EXCOUNTRY'].isin(usa_markets)]
financials_us.shape

# 342757 rows. Sounds right to me. But did I aslo include U.S. bonds? We will know soon

In [ ]:
len(financials_us['CUSIP'].unique())

# 8492. Seems very reasonable and also in line with what we saw in earlier data set

In [ ]:
institutions['key2'] = institutions['date_modified'] + institutions['CUSIP']

institutions['key2'][1:5]





# Joining the financials and institutions data sets

In [ ]:
financials_institutions_merged = pd.merge(financials_us,
                                          institutions,
                                          left_on='key1',
                                          right_on='key2',
                                          how='left')

In [ ]:
financials_institutions_merged.shape

In [ ]:
financials_institutions_merged.columns

# I just realized that common name columns are renamed as colname_x and colname_y. Thanks Wes Mckinney

In [ ]:
len(financials_institutions_merged['CUSIP_x'].unique())

In [ ]:
financials_institutions_merged['CUSIP_x'][1:5]

In [ ]:
len(financials_institutions_merged['CUSIP_y'].unique())

# I remember that the institution data set starts from 2000. However, the financial data set starts from 1990

In [ ]:
# So there are 8492 CUSIP's. That's great. But, for how many stocks do we have atleast 4 values (4 quarters is 1 year)
# If a stock hasn't traded for atleast a year, i think its best if I just drop that CUSIP right now

In [ ]:
CUSIP_count_df = financials_institutions_merged[['CUSIP_x','FS_PERM_SEC_ID']].groupby(['CUSIP_x']).agg(['count'])
CUSIP_count_df.columns = ['count']
CUSIP_count_df['CUSIP_stock'] = CUSIP_count_df.index

In [ ]:
len(CUSIP_count_df[CUSIP_count_df['count'] < 5])/len(CUSIP_count_df)

# I am so relieved to know that barely 5% of stocks in the data traded for less than a year. 






## Loading in the price and volume data set

In [ ]:
# loading in the monthly price and volume data set

price_volume = pd.read_csv("D:/Monthly price and volume.csv", encoding='latin-1')

In [ ]:
price_volume.shape

In [ ]:
price_volume[1:5]

In [ ]:
CUSIP_list = price_volume['CUSIP']
CUSIP_list = list(set(CUSIP_list))
len(CUSIP_list)

In [ ]:
# just of curiousity, how many unique PERM_NO are there?

PERMNO_list = price_volume['PERMNO']
PERMNO_list = list(set(PERMNO_list))
len(PERMNO_list)

# I know that 1 PERMNO can have multiple CUSIP. yet interestingly, both have same number of uniques

In [ ]:
# its interesting that there are nearly 23694 CUSIP's though. 

# Were there 23,000 stocks since 1990. I don't think so. If there were that many, I have the financials data only for roughly 8000
# My guess is that those extra 15000 stocks are OTC, ADR, candian or penny stocks etc. Let me confirm that

In [ ]:
price_volume.dtypes

# And that's why they say to set the type if column is too important!

In [ ]:
# The most important thing is to convert the entire monthly price volume data set into quarterly paradigm
# I am going to explore this data set deeply right now and not later because I really need to make sure values in original columns make sense before deriving other columns from old ones
# lets play with some columns to ensure I am manipualting them correctly when I do the conversion process

In [ ]:
# I noticed some values in numeric columns are strings. Impossible. I am just going to set them as NULL 
#I don't have the time to go through some special codes that occur once in a century

price_volume2 = price_volume[['VOL', 'ASKHI', 'PRC', 'BIDLO', 'RET', 'SHROUT', 'CFACPR', 'CFACSHR', 'vwretd', 'ewretd', 'sprtrn']]
no_strings = price_volume2.apply(pd.to_numeric, errors = 'coerce')

# quick note: its important that we don't run all the above blocks of code to successfully complete the above step
# I have a 4gb RAM and I get memory error if run the entire notebook

In [ ]:
price_volume = price_volume.drop(['VOL', 'ASKHI', 'PRC'], axis=1)
price_volume = price_volume.drop(['BIDLO'], axis=1)
price_volume = price_volume.drop(['RET', 'SHROUT'], axis=1)
price_volume = price_volume.drop(['CFACPR', 'CFACSHR', 'vwretd', 'ewretd', 'sprtrn'], axis=1)
xcv = price_volume
price_volume = pd.concat([xcv, no_strings], axis=1)

In [ ]:
# Before we do anything else, why don't we cut this data set to just the CUSIP codes for which we have financial data

CUSIPs_under_consideration = financials_institutions_merged['CUSIP_x'].unique()
sdf = price_volume[price_volume['CUSIP'].isin(CUSIPs_under_consideration)]

In [ ]:
sdf[1:5]

# no match between the CUSIP codes? wth?

In [ ]:
price_volume['CUSIP'][1:5]

In [ ]:
CUSIPs_under_consideration[1:5]

In [ ]:
CUSIPs_under_consideration[1000:1005]

In [ ]:
# The situation is clear. CUSIP's in price&volume are 8 digits. In the earlier merged file, they are 9 digits. What is happening?

# The best stock data base huh? No proper keys to join tables!

# Based on google search, it appears that i need to ignore the last digit of the 9 digit CUSIP

CUSIPs_under_consideration = pd.Series(CUSIPs_under_consideration)
CUSIPs_under_consideration = CUSIPs_under_consideration.str[0:8]

In [ ]:
price_volume = price_volume[price_volume['CUSIP'].isin(CUSIPs_under_consideration)]

In [ ]:
price_volume[1:5]

# it works!!!

In [ ]:
price_volume.shape

# almost 50% of the data is cut after selecting only certain CUSIP codes

In [53]:
# For most of the columns we only expect positive values.

# Broadly, for some of the columns a negative value or zero indicates something. So, how many special situations are there across columns
# For more specific details, refer to data dictionary

def bad_data(column_name):
    nulls = len(price_volume[price_volume[column_name].isnull()])*100/len(price_volume)
    zeros = len(price_volume[price_volume[column_name].astype(float) == 0])*100/len(price_volume)
    negatives = len(price_volume[price_volume[column_name].astype(float) < 0])*100/len(price_volume)
    special_situations = pd.Series([nulls, zeros, negatives])
    return special_situations

In [54]:
print(bad_data('VOL'))
print(bad_data('ASKHI'))
print(bad_data('PRC'))
print(bad_data('BIDLO'))
print(bad_data('RET'))
print(bad_data('SHROUT'))
print(bad_data('CFACPR'))
print(bad_data('CFACSHR'))
print(bad_data('vwretd'))
print(bad_data('ewretd'))
print(bad_data('sprtrn'))

0    0.000000
1    0.018213
2    0.000000
dtype: float64
0    0.0
1    0.0
2    0.0
dtype: float64
0    0.0
1    0.0
2    0.0
dtype: float64
0    0.0
1    0.0
2    0.0
dtype: float64
0     0.000000
1     1.622451
2    45.337307
dtype: float64
0    0.0
1    0.0
2    0.0
dtype: float64
0    0.000000
1    0.523423
2    0.000000
dtype: float64
0    0.000000
1    0.523423
2    0.000000
dtype: float64
0     0.000000
1     0.000000
2    35.445888
dtype: float64
0     0.000000
1     0.000000
2    38.205486
dtype: float64
0     0.000000
1     0.000000
2    36.928567
dtype: float64


In [ ]:
# There are only few negatives for most of the columns. Let me make them positive. I checked the data dictionary to make sure this makes sense

price_volume['BIDLO'] = abs(price_volume['BIDLO'])
price_volume['VOL'] = abs(price_volume['VOL'])
price_volume['ASKHI'] = abs(price_volume['ASKHI'])
price_volume['PRC'] = abs(price_volume['PRC'])
price_volume['SHROUT'] = abs(price_volume['SHROUT'])
price_volume['CFACPR'] = abs(price_volume['CFACPR'])
price_volume['CFACSHR'] = abs(price_volume['CFACSHR'])

# Oh god. Here we go again. Those copy of slice useless warnings. I don't know why the error is displayed only some times
# Don't worry about the warnings anyway

# I also noticed something else. SP500 increases daily more than 60% of days
# May be all I need to do is buy bullish binary options on NADEX everyday and call of this project. lol

In [ ]:
# But what are going to about the nulls though?

# Lets see how bad the null value problem is for each column

print(len(price_volume))

price_volume2 = price_volume
price_volume2 = price_volume[price_volume[['VOL', 'ASKHI', 'PRC', 'BIDLO', 'RET', 'SHROUT', 'CFACPR', 'CFACSHR', 'vwretd', 'ewretd', 'sprtrn']].isnull().any(axis=1)]
print(len(price_volume2))

# It's just 3% by no. of rows. 
# But its extremely important to look at what % of CUSIP codes we are going to lose because I need to delete all the rows beloning to the CUSIP to ensure there are no missing data for a particular CUSIP (stock)

In [ ]:
print(len(price_volume['CUSIP'].unique()))
print(len(price_volume2['CUSIP'].unique()))

# so basically, 7472 stocks had a null value at some point in time. Can't delete 7472 stocks obviously
# Need to think about this carefully

In [ ]:
# I want to look at the distribution of nulls. For example, are 50 stocks mostly responsible for causing the null problem?

null_CUSIP_count = pd.DataFrame(price_volume2.groupby(['CUSIP'])['PERMNO'].count())

In [ ]:
null_CUSIP_count['PERMNO'].describe()

# What is clear is that this a null problem that is spread across many stocks

In [ ]:
len(null_CUSIP_count[null_CUSIP_count['PERMNO'] > 3])

# But only 308 stocks have 4 or more null values. Why don't we just get rid of these 308 stocks

In [ ]:
CUSIP_remove = pd.Series(null_CUSIP_count[null_CUSIP_count['PERMNO'] > 3].index)

In [ ]:
CUSIP_remove[1:4]

In [ ]:
price_volume = price_volume[~price_volume['CUSIP'].isin(CUSIP_remove)]

In [37]:
# why don't we try impute now? Have to risk it. No option. Its only for less than 3 values anyway

price_volume = price_volume.fillna(method='ffill')

In [50]:
# But we are done yet. The nice people at WRDS have used special numeric codes to denote certain special events

# for example if ret contains -66 or -77, those values are not really -66 or -77. They are referring to some special situation

# I am going to check some numeric columns now to ensure that there are no crazy values. 

# I need to do something specifically about the zeros. Of course I am referring to the data dictionary and making sure the transformations I am about to do make sense

price_volume2 = price_volume[price_volume['VOL'] < 1000]

In [49]:
print(len(price_volume['CUSIP'].unique()))
print(len(price_volume))

7992
1180498


In [51]:
CUSIP_low_vol = price_volume2['CUSIP'].unique()
print(len(CUSIP_low_vol))
print(len(price_volume2))

# So just 30 rows per CUSIP on average for some CUSIP's. 30 months of data. also, so low volume. i don't want to deal with such data

# These are problem CUSIP's. Let me remove them

price_volume3 = price_volume[~price_volume['CUSIP'].isin(CUSIP_low_vol)]

3137
115606


In [59]:
RET = price_volume3.columns.get_loc("RET")

RET

16

In [63]:
median_RET = price_volume3['RET'].median()

for i in range(0,len(price_volume3)):
    if price_volume3.iloc[i,RET] == -66 | -77:
        price_volume3.iloc[i,RET] = median_RET
        
# because -66 and -77 are actually special codes

In [ ]:
print(bad_data('VOL'))
print(bad_data('ASKHI'))
print(bad_data('PRC'))
print(bad_data('BIDLO'))
print(bad_data('RET'))
print(bad_data('SHROUT'))
print(bad_data('CFACPR'))
print(bad_data('CFACSHR'))
print(bad_data('vwretd'))
print(bad_data('ewretd'))
print(bad_data('sprtrn'))

In [65]:
# The most important thing is to convert the entire monthly price volume data set into quarterly paradigm

all_stocks = price_volume3[1:1] # nice way to set all stocks df the columns of price_volume

for i in PERMNO_list[1:10]:
    individual_stock = price_volume3[price_volume3['PERMNO'] == i]
    
    vol_num = individual_stock.columns.get_loc("VOL")
    high_num = individual_stock.columns.get_loc("ASKHI")
    low_num = individual_stock.columns.get_loc("BIDLO")
    RET_num = individual_stock.columns.get_loc("RET")
    vwretd_num = individual_stock.columns.get_loc("vwretd")
    ewretd_num = individual_stock.columns.get_loc("ewretd")
    sprtrn_num = individual_stock.columns.get_loc("sprtrn")
    
    individual_stock['three_month_vol'] = None
    individual_stock['three_month_high'] = None
    individual_stock['three_month_low'] = None
    individual_stock['three_month_RET'] = None
    individual_stock['three_month_vwretd'] = None
    individual_stock['three_month_ewretd'] = None
    individual_stock['three_month_sprtrn'] = None
    
    three_month_vol = individual_stock.columns.get_loc("three_month_vol")
    three_month_high = individual_stock.columns.get_loc("three_month_high")
    three_month_low = individual_stock.columns.get_loc("three_month_low")
    three_month_RET = individual_stock.columns.get_loc("three_month_RET")
    three_month_vwretd = individual_stock.columns.get_loc("three_month_vwretd")
    three_month_ewretd = individual_stock.columns.get_loc("three_month_ewretd")
    three_month_sprtrn = individual_stock.columns.get_loc("three_month_sprtrn")
    
    individual_stock.iloc[0,three_month_vol] = individual_stock.iloc[0, vol_num]*3
    individual_stock.iloc[1,three_month_vol] = individual_stock.iloc[1, vol_num]*3
    individual_stock.iloc[0,three_month_high] = individual_stock.iloc[0, high_num]*3
    individual_stock.iloc[1,three_month_high] = individual_stock.iloc[1, high_num]*3
    individual_stock.iloc[0,three_month_low] = individual_stock.iloc[0, low_num]*3
    individual_stock.iloc[1,three_month_low] = individual_stock.iloc[1, low_num]*3
    individual_stock.iloc[0,three_month_RET] = individual_stock.iloc[0, RET_num]*3
    individual_stock.iloc[1,three_month_RET] = individual_stock.iloc[1, RET_num]*3
    individual_stock.iloc[0,three_month_vwretd] = individual_stock.iloc[0, vwretd_num]*3
    individual_stock.iloc[1,three_month_vwretd] = individual_stock.iloc[1, vwretd_num]*3
    individual_stock.iloc[0,three_month_ewretd] = individual_stock.iloc[0, ewretd_num]*3
    individual_stock.iloc[1,three_month_ewretd] = individual_stock.iloc[1, ewretd_num]*3
    individual_stock.iloc[0,three_month_sprtrn] = individual_stock.iloc[0, sprtrn_num]*3
    individual_stock.iloc[1,three_month_sprtrn] = individual_stock.iloc[1, sprtrn_num]*3
    
    
    for j in range(2,len(individual_stock)):
        individual_stock.iloc[j,three_month_vol] = individual_stock.iloc[j, vol_num] + individual_stock.iloc[j-1, vol_num] + individual_stock.iloc[j-2, vol_num]
        individual_stock.iloc[j,three_month_high] = max(individual_stock.iloc[j, high_num], individual_stock.iloc[j-1, high_num], individual_stock.iloc[j-2, high_num])
        individual_stock.iloc[j,three_month_low] = min(individual_stock.iloc[j, low_num], individual_stock.iloc[j-1, low_num], individual_stock.iloc[j-2, low_num])
        individual_stock.iloc[j,three_month_RET] = (1 + individual_stock.iloc[j,RET_num])(1 + individual_stock.iloc[j-1,RET_num])(1 + individual_stock.iloc[j-2,RET_num])
        individual_stock.iloc[j,three_month_vwretd] = (1 + individual_stock.iloc[j,vwretd_num])(1 + individual_stock.iloc[j-1,vwretd_num])(1 + individual_stock.iloc[j-2,vwretd_num])
        individual_stock.iloc[j,three_month_vwretd] = (1 + individual_stock.iloc[j,ewretd_num])(1 + individual_stock.iloc[j-1,ewretd_num])(1 + individual_stock.iloc[j-2,ewretd_num])
        individual_stock.iloc[j,three_month_sprtrn] = (1 + individual_stock.iloc[j,sprtrn_num])(1 + individual_stock.iloc[j-1,sprtrn_num])(1 + individual_stock.iloc[j-2,sprtrn_num])
        
        
    all_stocks = all_stocks.append(individual_stock, ignore_index=True)    
    
# ignore the warnings below. Its the typical 'a value being set on slice etc.' 

C:\Users\Prasanth\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Prasanth\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [66]:
all_stocks[1:10]

,ASKHI,BIDLO,CFACPR,CFACSHR,COMNAM,CUSIP,NWPERM,PERMCO,PERMNO,PRC,...,SHROUT,SICCD,TRDSTAT,TSYMBOL,VOL,date,ewretd,sprtrn,three_month_vol,vwretd
1,24.000,21.75,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,24.00,...,13596.0,3560,A,FIGIA,9407.0,02/28/1990,0.015434,0.008539,28221.0,0.014901
2,24.250,22.75,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,23.75,...,13590.0,3560,A,FIGIA,11441.0,03/30/1990,0.021315,0.024255,25140.0,0.024140
3,24.000,22.75,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,22.75,...,13590.0,3560,A,FIGIA,6909.0,04/30/1990,-0.028116,-0.026887,27757.0,-0.028286
4,25.000,22.75,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,24.25,...,13590.0,3560,A,FIGIA,13678.0,05/31/1990,0.045673,0.091989,32028.0,0.088934
5,25.500,23.75,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,24.75,...,13593.0,3560,A,FIGIA,10995.0,06/29/1990,0.005359,-0.008886,31582.0,-0.004196
6,25.125,23.00,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,23.00,...,13593.0,3560,A,FIGIA,8946.0,07/31/1990,-0.025731,-0.005223,33619.0,-0.009403
7,23.750,16.50,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,17.50,...,13593.0,3560,A,FIGIA,7365.0,08/31/1990,-0.108920,-0.094314,27306.0,-0.091905
8,18.000,15.50,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,15.50,...,13593.0,3560,A,FIGIA,6291.0,09/28/1990,-0.080468,-0.051184,22602.0,-0.053843
9,15.500,10.50,1.0,1.0,FIGGIE INTERNATIONAL INC DEL,81002230,0.0,1728,10016,12.00,...,13593.0,3560,A,FIGIA,15200.0,10/31/1990,-0.057155,-0.006698,28856.0,-0.012504


In [80]:
financials_institutions_merged['CUSIP_cut'] = financials_institutions_merged['CUSIP_x'].str[0:8]

In [83]:
financials_institutions_pricevolume =  pd.merge(financials_institutions_merged,
                                                all_stocks,
                                                left_on='CUSIP_cut',
                                                right_on='CUSIP',
                                                how='inner')

In [84]:
financials_institutions_pricevolume[1:4]

,FS_PERM_SEC_ID,CUSIP_x,TIC_x,EXCOUNTRY,PRIMARY_SIC_CODE,ZIP_POSTAL_CODE,FF_COMPACT,FF_MKT_VAL_CURR,FF_IS_ADR,FF_SECACT,...,SHROUT,SICCD,TRDSTAT,TSYMBOL,VOL,date,ewretd,sprtrn,three_month_vol,vwretd
1,G0Z82X-S-US,810022301,SCTT.XX1,NAS,2874.0,43041,0,NaN,0,0,...,13596.0,3560,A,FIGIA,9407.0,02/28/1990,0.015434,0.008539,28221.0,0.014901
2,G0Z82X-S-US,810022301,SCTT.XX1,NAS,2874.0,43041,0,NaN,0,0,...,13590.0,3560,A,FIGIA,11441.0,03/30/1990,0.021315,0.024255,25140.0,0.024140
3,G0Z82X-S-US,810022301,SCTT.XX1,NAS,2874.0,43041,0,NaN,0,0,...,13590.0,3560,A,FIGIA,6909.0,04/30/1990,-0.028116,-0.026887,27757.0,-0.028286



# Loading in the 13F data set

In [92]:
fund13f = pd.read_csv("D:/13F.csv", encoding='latin-1', nrows = 6000000)

# its really mysterious. When I use nrows arguement, its able to read in 6,000,000 rows in less than 30 seconds
# However, when I do just read_csv, my pc's getting stuck. I know that there are 6.7 million rows in this file anyway

In [ ]:
fund13f.shape

# 6.7 million rows. This data set is much bigger than the previous 2 data sets. But most importantly, it is possible to use columns to join this data set

In [93]:
fund13f.columns

Index(['FACTSET_ENTITY_ID', 'FSYM_ID', 'REPORT_DATE', 'ADJ_MV',
       'REPORTED_HOLDING'],
      dtype='object')

In [ ]:
len(fund13f['FACTSET_ENTITY_ID'].unique())

In [ ]:
len(fund13f['FSYM_ID'].unique())

# I thought FSYM_ID is the unique key for stocks. But there aren't 59,980 stocks. May be FACTSET_ENTITY_ID is the unique key for stocks

In [ ]:
fund13f['FACTSET_ENTITY_ID'][1:5]

In [ ]:
fund13f['FSYM_ID'][1:5]

In [ ]:
financials_institutions_merged['FS_PERM_SEC_ID'][1:5]

In [ ]:
# verified on WRDS forums. FACTSET_ENTITY_ID is the actual unique key for stocks. Is it referred by a different name in the earlier merged data set

stocks_code1 = fund13f['FACTSET_ENTITY_ID']
stocks_code2 = financials_institutions_merged['FS_PERM_SEC_ID']

In [ ]:
common_codes1 = pd.Series(list(set(stocks_code1).intersection(set(stocks_code2))))

len(common_codes1)

# But there are no common codes. I think the financials_institutions_merged codes have '-US' at the end. 

In [94]:
fund13f['New_FSYM_ID'] = fund13f['FSYM_ID'] + '-US'

In [ ]:
fund13f[1:5]

In [ ]:
fund13f_stock_codes = fund13f2['New_FSYM_ID']
earlier_merged_stock_codes = financials_institutions_merged['FS_PERM_SEC_ID']

common_codes = pd.Series(list(set(fund13f_stock_codes).intersection(set(earlier_merged_stock_codes))))
len(common_codes)

# yup. Adding 'US' did the trick

# Read the research papers written by Professor Mateos who developed the institutional data set. It appears he is focused on corporate finance related metrics rather qunatitative asset management (or predicting stock market)

In [ ]:
# Now that we can use the 13F data set, the question is how do I calculate 'how good a fund is?'

In [95]:
fund13f.columns

Index(['FACTSET_ENTITY_ID', 'FSYM_ID', 'REPORT_DATE', 'ADJ_MV',
       'REPORTED_HOLDING', 'New_FSYM_ID'],
      dtype='object')

In [139]:
fund13f.isnull().sum()

# Great the most useful column has 45143 nulls

FACTSET_ENTITY_ID        0
FSYM_ID                  0
REPORT_DATE              0
ADJ_MV               45143
REPORTED_HOLDING         0
New_FSYM_ID              0
date_combo               0
to_be_joined             0
dtype: int64

In [152]:
null_index = fund13f[fund13f['ADJ_MV'].isnull()].index

In [157]:
fund13f.loc[null_index][0:5]

,FACTSET_ENTITY_ID,FSYM_ID,REPORT_DATE,ADJ_MV,REPORTED_HOLDING,New_FSYM_ID,date_combo,to_be_joined
616,000BJX-E,CSTD7Y-S,03/31/2002,NaN,46400,CSTD7Y-S-US,20020331,03/31/2002CSTD7Y-S-US
617,000BJX-E,CSTD7Y-S,06/30/2002,NaN,46400,CSTD7Y-S-US,20020630,06/30/2002CSTD7Y-S-US
618,000BJX-E,CSTD7Y-S,09/30/2002,NaN,46400,CSTD7Y-S-US,20020930,09/30/2002CSTD7Y-S-US
8054,000BKX-E,C5J6CK-S,06/30/2001,NaN,314602,C5J6CK-S-US,20010630,06/30/2001C5J6CK-S-US
8055,000BKX-E,C5J6CK-S,09/30/2001,NaN,314603,C5J6CK-S-US,20010930,09/30/2001C5J6CK-S-US


In [158]:
# Before we spend further time on this, first question: how many New_FSYM ID's have this null problem?

fund13f_null = fund13f.loc[null_index]

In [159]:
len(fund13f_null['New_FSYM_ID'].unique())

# So, its a big problem. More than 10% of funds

7451

In [160]:
print(len(fund13f))
print(len(fund13f_null))

# So, on average these problem hedge funds have made an appearance for just 6 - 7 quarters. I say delete them

6000000
45143


In [161]:
fund13f = fund13f.dropna()

In [162]:
fund13f.isnull().sum()

# great

FACTSET_ENTITY_ID    0
FSYM_ID              0
REPORT_DATE          0
ADJ_MV               0
REPORTED_HOLDING     0
New_FSYM_ID          0
date_combo           0
to_be_joined         0
dtype: int64

In [163]:
total_portfolio = fund13f.groupby(['REPORT_DATE','New_FSYM_ID']).sum()

In [164]:
total_portfolio[1:5]

ADJ_MV  REPORTED_HOLDING
REPORT_DATE New_FSYM_ID                               
03/31/1999  B00GXG-S-US   65125693.0           4938254
            B00L3K-S-US    4942009.0           1040423
            B01573-S-US  294315326.0          12901211
            B01DPB-S-US  106733801.0           3712480

In [176]:
total_portfolio2 = total_portfolio[['REPORT_DATE','New_FSYM_ID','ADJ_MV']]

In [177]:
total_portfolio2[1:5]

,REPORT_DATE,New_FSYM_ID,ADJ_MV
1,03/31/1999,B00GXG-S-US,65125693.0
2,03/31/1999,B00L3K-S-US,4942009.0
3,03/31/1999,B01573-S-US,294315326.0
4,03/31/1999,B01DPB-S-US,106733801.0


In [185]:
total_portfolio2['to_be_joined_key2'] = total_portfolio2['REPORT_DATE'] + total_portfolio2['New_FSYM_ID']

In [183]:
fund13f['to_be_joined'] = fund13f['REPORT_DATE'] + fund13f['New_FSYM_ID']

In [187]:
fund13f2 = pd.merge(fund13f,
                    total_portfolio2,
                    left_on='to_be_joined',
                    right_on='to_be_joined_key2',
                    how='left')

In [188]:
fund13f2[0:5]

,FACTSET_ENTITY_ID,FSYM_ID,REPORT_DATE_x,ADJ_MV_x,REPORTED_HOLDING,New_FSYM_ID_x,date_combo,to_be_joined,REPORT_DATE_y,New_FSYM_ID_y,ADJ_MV_y,to_be_joined_key2
0,000BJX-E,B00CQC-S,12/31/2004,534527.0,13300,B00CQC-S-US,20041231,12/31/2004B00CQC-S-US,12/31/2004,B00CQC-S-US,1.004516e+09,12/31/2004B00CQC-S-US
1,000BJX-E,B00CQC-S,03/31/2005,390040.0,9800,B00CQC-S-US,20050331,03/31/2005B00CQC-S-US,03/31/2005,B00CQC-S-US,1.039999e+09,03/31/2005B00CQC-S-US
2,000BJX-E,B00CQC-S,06/30/2005,390299.0,9100,B00CQC-S-US,20050630,06/30/2005B00CQC-S-US,06/30/2005,B00CQC-S-US,1.094276e+09,06/30/2005B00CQC-S-US
3,000BJX-E,B00CQC-S,09/30/2005,417417.0,9100,B00CQC-S-US,20050930,09/30/2005B00CQC-S-US,09/30/2005,B00CQC-S-US,1.242329e+09,09/30/2005B00CQC-S-US
4,000BJX-E,B00CQC-S,12/31/2005,188225.0,4350,B00CQC-S-US,20051231,12/31/2005B00CQC-S-US,12/31/2005,B00CQC-S-US,1.029819e+09,12/31/2005B00CQC-S-US


In [191]:
fund13f2['date_combo'] = fund13f2['REPORT_DATE_x'].astype(str)
fund13f2['date_combo'] = fund13f2.REPORT_DATE_x.str[6:10] + fund13f2.REPORT_DATE_x.str[0:2] + fund13f2.REPORT_DATE_x.str[3:5]

In [192]:
fund13f2['date_combo'][1:5]

# date_combo is going to be the ultimate column that we are going to use to join

1    20050331
2    20050630
3    20050930
4    20051231
Name: date_combo, dtype: object

In [193]:
fund13f2.columns

Index(['FACTSET_ENTITY_ID', 'FSYM_ID', 'REPORT_DATE_x', 'ADJ_MV_x',
       'REPORTED_HOLDING', 'New_FSYM_ID_x', 'date_combo', 'to_be_joined',
       'REPORT_DATE_y', 'New_FSYM_ID_y', 'ADJ_MV_y', 'to_be_joined_key2'],
      dtype='object')

In [194]:
fund13f2 = fund13f2.rename(index=str, columns={"ADJ_MV_y": "Portfolio_value"})

MemoryError: 

In [ ]:
fund13f2['Percentage_of_fund'] = fund13f2['ADJ_MV_x']/fund13f2['Portfolio_value']

In [ ]:
fund13f2 = fund13f2[['date_combo', 'FACTSET_ENTITY_ID', 'New_FSYM_ID_x', '']]

In [117]:
financials_institutions_pricevolume['FS_PERM_SEC_ID'][0:4]

0    G0Z82X-S-US
1    G0Z82X-S-US
2    G0Z82X-S-US
3    G0Z82X-S-US
Name: FS_PERM_SEC_ID, dtype: object

In [ ]:
fund13f3 = pd.merge(fund13f2,
                    financials_institutions_pricevolume['RET'],
                    left_on = 'to_be_joined',
                    right_on = 'RET',
                    how='left')